Import Libaries

In [ ]:
import pandas as pd
import sklearn
import nltk
import string
import re
#from textblob import TextBlob
import langid
from tqdm import tqdm
tqdm.pandas()


Read CSV with pandas

In [ ]:
'''
trump_tweets = pd.read_csv('hashtag_donaldtrump.csv',lineterminator='\n')
trump_tweets = trump_tweets.drop(   ['country'
                                    ,'continent'
                                    ,'state'
                                    ,'user_name'
                                    ,'user_id'
                                    ,'user_screen_name'
                                    ,'tweet_id'
                                    ,'lat'
                                    ,'long'
                                    ,'city'
                                    ,'state_code'
                                    ,'source'
                                    ,'collected_at'
                                    ,'user_followers_count'
                                    ,'user_location'
                                    ,'user_description'
                                    ,'user_join_date']
                                    ,axis = 1)

biden_tweets = pd.read_csv('hashtag_joebiden.csv',lineterminator='\n')
biden_tweets = biden_tweets.drop(   ['country'
                                    ,'continent'
                                    ,'state'
                                    ,'user_name'
                                    ,'user_id'
                                    ,'user_screen_name'
                                    ,'tweet_id'
                                    ,'lat'
                                    ,'long'
                                    ,'city'
                                    ,'state_code'
                                    ,'source'
                                    ,'collected_at'
                                    ,'user_followers_count'
                                    ,'user_location'
                                    ,'user_description'
                                    ,'user_join_date']
                                    ,axis = 1)
'''
# print('#Trump Tweets:\n',trump_tweets.head())
# print(len(trump_tweets),trump_tweets.shape)
# print("#Biden Tweets:\n",biden_tweets.head())
# print(biden_tweets.shape)

Get Tweets for dates

In [ ]:
#date format ddmmyyyy
def get_tweets_dates(df,start_date,stop_date):
    ls = []
    start_month = int(start_date[2:4])
    stop_month = int(stop_date[2:4])
    start_day = int(start_date[0:2])
    stop_day = int(stop_date[0:2])
    gen = df.itertuples()
    for i in gen:
        tweet_date = i[1]
        tweet_month = int(tweet_date[5:7])
        tweet_day = int(tweet_date[8:10])
        #print(tweet_month,tweet_day)
        if tweet_month >= start_month and tweet_month <= stop_month:
            if tweet_month == start_month and tweet_day < start_day:
                pass
            elif tweet_month == stop_month and tweet_day > stop_day:
                pass
            else:
                ls.append(i)
    return pd.DataFrame(ls)
# test = get_tweets_dates(biden_tweets,'20102020','20102020')
# print(test.iloc[0])
# print(test.iloc[-1])

Trump: oct:[15-31],nov:[1-8]: 25

Biden: Oct[15-31],nov:[1-8] : 25

In [ ]:
ls_oct_dates = [str(i)+'102020' for i in range(15,32)]
ls_nov_dates = ['0'+str(i)+'112020' for i in range(1,9)]
ls_dates = ls_oct_dates+ls_nov_dates

def get_dict_tweets_dates(ls_dates,df):
    re_dd = {}
    for i in ls_dates:
        re_dd[i] = get_tweets_dates(df,i,i)
    return re_dd

#trump_tweets_by_day_dd = get_dict_tweets_dates(ls_dates,trump_tweets)
#biden_tweets_by_day_dd = get_dict_tweets_dates(ls_dates,biden_tweets)

Helper Fucntions for Cleaning Text

In [46]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

def remove_whitespace(text):
    text = re.sub(" +", " ", text)
    return text

def remove_newline(text):
    text = text.replace("\n", "")
    text = text.replace("\t", "")
    return text

def remove_urls(text):
    ls = text.split()
    re_str = ''
    for i in ls:
        if 'https' in i:
            pass
        else:
            re_str+=' '+i
    return re_str
def filter_english(text):
    test = langid.classify(text)[0]
    # ('en', -54.41310358047485)
    # sleep(100)
    if test == 'en':
        return text
    else:
        return '### NON-ENGLISH LANGUAGE ###'
def clean_everything(text):
    text = str(text)
    text = remove_urls(text)
    text = remove_newline(text)
    text = remove_punctuations(text)
    text = remove_whitespace(text)
    text = filter_english(text)
    return text

def clean_tweet_df(df):
    df['tweet'] = df['tweet'].progress_apply(lambda x: clean_everything(x))
    return df

def clean_tweet_df_from_dd(df_dd):
    re_dd = {}
    keys = df_dd.keys()
    for i in keys:
        df = df_dd[i]
        re_dd[i] = clean_tweet_df(df)
    return re_dd

def remove_non_english_rows_from_df(df):
    df = df.drop(df[df.tweet == '### NON-ENGLISH LANGUAGE ###'].index)
    df = df.drop(df[df.tweet == 'NON-ENGLISH LANGUAGE'].index)
    df = df.drop(df[df.tweet == ' NON-ENGLISH LANGUAGE '].index)
    df = df.drop(df[df.tweet == ' NON-ENGLISH LANGUAGE'].index)
    df = df.drop(df[df.tweet == 'NON-ENGLISH LANGUAGE '].index)
    df = df.drop(df[df.tweet == '### NON ENGLISH LANGUAGE ###'].index)
    df = df.drop(df[df.tweet == 'NON ENGLISH LANGUAGE'].index)
    df = df.drop(df[df.tweet == ' NON ENGLISH LANGUAGE '].index)
    df = df.drop(df[df.tweet == ' NON ENGLISH LANGUAGE'].index)
    df = df.drop(df[df.tweet == 'NON ENGLISH LANGUAGE '].index)
    return df

print('before cleaning: ', 'ni hao mah')
print('after cleaning: ', clean_everything('ni hao mah'))

before cleaning:  ni hao mah
after cleaning:  ### NON-ENGLISH LANGUAGE ###


Clean Each DataFrame and save to csv

In [ ]:
#trump_tweets_by_day_dd = clean_tweet_df_from_dd(trump_tweets_by_day_dd)
#biden_tweets_by_day_dd = clean_tweet_df_from_dd(biden_tweets_by_day_dd)
#print('Done Cleaning')

In [ ]:
def save_to_csvs_from_dd(df_dd,name):
    keys = df_dd.keys()
    for i in keys:
        df = df_dd[i]
        df.to_csv(f'tweets_by_day/{i}_{name}.csv',index = False)

# save_to_csvs_from_dd(biden_tweets_by_day_dd,'biden')
# save_to_csvs_from_dd(trump_tweets_by_day_dd,'trump')

Collect data from csvs

In [47]:
def get_dd_from_csv_folder(foldername,name):
    re_dd = {}
    for i in os.listdir(foldername):
        #print(i)
        if i[9:-4] == name:
            df = pd.read_csv(foldername+'/'+i)
            #print(i[0:8])
            re_dd[i[0:8]] = remove_non_english_rows_from_df(df)
        else:
            pass
    return re_dd
biden_tweets_by_day_dd = get_dd_from_csv_folder('tweets_by_day','biden')
trump_tweets_by_day_dd = get_dd_from_csv_folder('tweets_by_day','trump')
print('\nDone!')
print(trump_tweets_by_day_dd['27102020']['tweet'])


Done!
2         Had to be same success as Trump s casino busi...
3         With the COVID 19 pandemic dragging on and th...
4         realDonaldTrump BREAKING COVID 19 OUTBREAK IN...
6         CapehartJ This may be a rhetorical question b...
7         Heads up AmyComeyBarrett if ClarenceThomas me...
                               ...                        
22976          trump2020 OperationMAGA Trump MAGA2020 maga
22977     DonaldTrump realDonaldTrump SenTedCruz Saving...
22979     Because the Trump Campaign are Liars and Liar...
22980     therecount Yes we do That s what ypur lying b...
22981     trump wants to make it illegal to cover COVID...
Name: tweet, Length: 17829, dtype: object


Get the TF-IDF for the tweets

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf(df):
    doc=df['tweet'].values #df is your dataframe
    #print(doc)
    tf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tf.fit_transform(doc)
    feature_names = tf.get_feature_names()
    doc = 0
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    ls = []
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
        ls.append((w,s))
    return ls

def from_dd_to_tfidf(dd,file_name):
    keys = dd.keys()
    ls = []
    for i in keys:
        test_ls = tf_idf(remove_non_english_rows_from_df(dd[i]))
        test_ls.insert(0,(i,i))
        ls=ls+test_ls
    csv_df = pd.DataFrame(ls)
    csv_df.to_csv(f'tweets_by_day_tfidf/tfidf_{file_name}.csv',index=False)
    return ls

from_dd_to_tfidf(biden_tweets_by_day_dd,'biden')
from_dd_to_tfidf(trump_tweets_by_day_dd,'trump')

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
def get_top_k_retweets_daily(dd,k,filename):
    keys = dd.keys()
    re_dd = {}
    try:
        os.mkdir(f'top_{k}_retweets_by_day')
    except:
        print('oop')
    for i in keys:
        df = remove_non_english_rows_from_df(dd[i])
        top_k = df.nlargest(k,'retweet_count')
        re_dd[i] = top_k
        top_k.to_csv(f'top_{k}_retweets_by_day/{i}_{filename}.csv',index = False)
    return re_dd

def get_top_k_likes_daily(dd,k,filename):
    keys = dd.keys()
    re_dd = {}
    try:
        os.mkdir(f'top_{k}_likes_by_day')
    except:
        print('oop')
    for i in keys:
        df = remove_non_english_rows_from_df(dd[i])
        top_k = df.nlargest(k,'likes')
        re_dd[i] = top_k
        top_k.to_csv(f'top_{k}_likes_by_day/{i}_{filename}.csv',index = False)
    return re_dd



get_top_k_retweets_daily(trump_tweets_by_day_dd,5,'trump')
get_top_k_retweets_daily(biden_tweets_by_day_dd,5,'biden')
get_top_k_likes_daily(trump_tweets_by_day_dd,5,'trump')
get_top_k_likes_daily(biden_tweets_by_day_dd,5,'biden')